In [2]:
# Import the necessary modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl
mpl.rc('figure', figsize=(8, 7))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')

# Magic to tell python notebook we want matplotlib charts included
%matplotlib inline

In [3]:
train_df = pd.read_json("training_set.json")
test_df = pd.read_json("test_set.json")
# hashtag_corpus = pd.read_json("NTUSDFinCorpus/NTUSD_Fin_hashtag_v1.0.json")
# word_corpus = pd.read_json("NTUSDFinCorpus/NTUSD_Fin_word_v1.0.json")

In [4]:
conditions = [
    (train_df['sentiment'] == 0) , 
    (train_df['sentiment'] <  0) ,
    (train_df['sentiment'] >  0)]
choices = ['neutral', 'bullish', 'bearish']
train_df['classes'] = np.select(conditions, choices, default='neutral')

# Pre-Processing

In [5]:
train_df['tweet'] = train_df['tweet'].str.lower()
train_df['tweet'] = train_df['tweet'].str.replace('([@][\w_-]+)', ' @mentions ', case=False)
train_df['tweet'] = train_df['tweet'].str.replace('([$][a-z]+)', ' @cashtag ', case=False)
train_df['tweet'] = train_df['tweet'].str.replace('http\S+|www.\S+', '@url', case=False)

In [ ]:
def 

In [6]:
train_df

,sentiment,snippet,target,tweet,classes
0,-0.463,downgrade,$PCAR,downgrades @cashtag @cashtag @cashtag @...,bullish
1,0.678,"[looking sexy this morning, break on volume]",$AMZN,@cashtag looking sexy this morning...$600 br...,bearish
2,0.377,still long term fan!,$SBUX,@mentions stock hasn't moved much since firs...,bearish
3,0.129,$TFM will have a way to go price wise to compe...,$KR,whole foods @cashtag may feel price competit...,bearish
4,0.395,iPhone SE Could Be Doing Better Than Expected,$AAPL,apple's iphone se could be doing better than e...,bearish
5,0.458,Now up to 200 stocks making new 52 week highs,$TJX,now up to 200 stocks making new 52 week highs ...,bearish
6,0.288,There's no reason why $GOOGL can't get back to...,$GOOGL,there's no reason why @cashtag can't get bac...,bearish
7,-0.453,short,$CVEO,nice names on watch for tomorrow @cashtag lo...,bullish
8,0.372,Pivotal sees 27% upside for Alphabet,$YHOO,pivotal sees 27% upside for alphabet @url @ca...,bearish
9,0.216,the most trending,$DWTI,good morning! here's some of the most trending...,bearish


# Training the Model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer( analyzer='word',
                      ngram_range=(1,1),
                      stop_words = 'english')

X = list(train_df["tweet"])
y = train_df["sentiment"]

X = cv.fit_transform(X)

In [ ]:
model = cv.fit_transform(list(train_df["tweet"]))

In [10]:
# nothing (1396, 4608)
# smth (1396, 3249)
print(X.shape)

(1396, 3030)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.33, random_state=42 )

In [12]:
#linear regression model
from sklearn.linear_model import LinearRegression
log_model = LinearRegression()
log_model = log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)

#random forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

print(clf.feature_importances_)
forest_prediction = clf.predict(X_test)
print(forest_prediction)
print(mean_squared_error(y_test, forest_prediction))

predictions = nb.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test, predictions))

In [15]:
y_pred

array([ -5.87302458e-01,   7.30000013e-02,   8.65339229e-01,
        -2.46433207e-01,  -2.81947619e-01,   4.51999980e-01,
        -4.00499997e-01,   1.23441694e-01,  -4.42999981e-01,
         1.06928689e-01,  -1.35999975e-01,  -2.73868925e-02,
         4.47082817e-01,   5.06180880e-02,   4.81861774e-01,
        -2.40425830e-01,   9.68995979e-02,   3.52000024e-01,
         1.23078658e-01,   4.42541768e-01,   5.11953393e-01,
         3.65000020e-01,   2.77629527e-01,   1.17588824e-01,
         7.30000013e-02,  -2.20907678e-01,   1.42760343e-01,
         3.99666665e-01,   6.14285181e-01,   6.63006849e-01,
         2.83200005e-01,   1.85681039e-01,  -4.43251096e-01,
        -2.06166008e-02,   3.23277038e-01,  -2.56868866e-01,
         5.99566842e-01,   1.15578648e-01,   1.65186584e-01,
         2.42085955e-01,   1.59381572e-01,   3.06365966e-01,
         4.61000013e-01,  -2.91664380e-01,   1.65000026e-01,
         9.89387538e-02,  -2.41148518e-02,  -1.86038725e-01,
        -4.08359292e-01,

# Evaluation

In [17]:
def assignClasses(data): 
    value = list()
    for i in data: 
        if i > 0:
            value.append("bullish")
        elif i < 0:
            value.append("bearish")
        else:
            value.append("neutral")
    
    return value 

In [21]:
n_y_test = assignClasses(y_test)
n_y_pred = assignClasses(y_pred)

In [26]:
len(n_y_test)

461

In [23]:
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print(mean_squared_error(y_test, y_pred))
print('\n')
f1_score(n_y_test, n_y_pred, average='macro')
f1_score(n_y_test, n_y_pred, average='micro')


0.328189489997




D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.67462039045553146